In [1]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# 🚀 Carregar o modelo
model = SentenceTransformer('all-mpnet-base-v2')

# 🚀 Carregar o DataFrame e os embeddings
dataset = pd.read_csv('./data/processed/DATASET_FINAL_EMBEDDINGS.csv')

with open('./data/processed/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

# 🚀 Normalizando os embeddings para otimizar a busca
embeddings = np.array(embeddings).astype('float32')
faiss.normalize_L2(embeddings)

# 🚀 Criar o índice FAISS
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

print(f"✅ Índice de busca criado com {index.ntotal} jurisprudências indexadas.")

# 🔍 Função de busca semântica
def buscar_jurisprudencia(query, top_k=5):
    # Gerar o embedding da consulta
    query_embedding = model.encode([query], normalize_embeddings=True)
    
    # Realizar a busca
    distances, indices = index.search(np.array(query_embedding).astype('float32'), top_k)
    
    resultados = []
    for idx, score in zip(indices[0], distances[0]):
        resultado = {
            'diagnóstico': dataset.loc[idx, 'diagnóstico'],
            'conclusão': dataset.loc[idx, 'conclusão'],
            'conclusão justificada': dataset.loc[idx, 'conclusão justificada'],
            'cid': dataset.loc[idx, 'cid'],
            'princípio ativo': dataset.loc[idx, 'princípio ativo'],
            'nome comercial': dataset.loc[idx, 'nome comercial'],
            'tipo da tecnologia': dataset.loc[idx, 'tipo da tecnologia'],
            'órgão': dataset.loc[idx, 'órgão'],
            'serventia': dataset.loc[idx, 'serventia'],
            'referência': dataset.loc[idx, 'link visualização'],
            'similaridade': float(score)
        }
        resultados.append(resultado)

    return pd.DataFrame(resultados)

# 🚀 Exemplo de Consulta
print("🔎 Exemplo de Busca:")
resultado_busca = buscar_jurisprudencia("Qual o entendimento sobre o Obesidade em Goiás?", top_k=5)
print(resultado_busca)


c:\Users\mlzengo\Documents\TJGO\SPRINT IV\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Índice de busca criado com 1708 jurisprudências indexadas.
🔎 Exemplo de Busca:
  diagnóstico                                          conclusão  \
0  Obesidade.  1. Trata-se de demanda judicial envolvendo o r...   
1  Obesidade.  1. Trata-se de demanda judicial envolvendo o r...   
2  Obesidade.  Considerando quadro clínico da requerente desc...   
3  Obesidade.  1. Trata-se de demanda judicial envolvendo a r...   
4   Obesidade  Considerando o diagnóstico informado de obesid...   

  conclusão justificada                                 cid princípio ativo  \
0         Não favorável                     E66 - Obesidade   Não Informado   
1             Favorável  E66.9 - Obesidade não especificada   Não Informado   
2             Favorável                     E66 - Obesidade     LIRAGLUTIDA   
3             Favorável                     E66 - Obesidade   Não Informado   
4             Favorável                     E66 - Obesidade     LIRAGLUTIDA   

  nome comercial tipo da tecnologia